<a href="https://colab.research.google.com/github/hr1588/dacon/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2022-08-11 13:07:47--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2022-08-11 13:07:47--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 142.251.6.101, 142.251.6.139, 142.251.6.100, ...
Connecting to drive.google.com (drive.google.com)|142.251.6.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/euf6ga102vqon2ld85l3rvf7kkoc5tm5/1660223250000/17946651057176172524/*/1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw?e=download&uuid=c9efe8d3-9d1a-40ab-aeae-16294d154867 [following]
--2022-08-11 13:07:48--  https://doc-10-10-docs.google

In [2]:
# 라이브러리 불러오기
import seaborn as sns
import pandas as pd

# VIF기능을 제공하는 라이브러리 불러오기
from statsmodels.stats.outliers_influence import variance_inflation_factor

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
train = pd.read_csv('data/train.csv')

# 범주형 변수 제거(type)
train.drop('type',inplace = True,axis=1)

In [4]:
train

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5492,5492,5,7.7,0.150,0.29,1.3,0.029,10.0,64.0,0.99320,3.35,0.39,10.1
5493,5493,6,6.3,0.180,0.36,1.2,0.034,26.0,111.0,0.99074,3.16,0.51,11.0
5494,5494,7,7.8,0.150,0.34,1.1,0.035,31.0,93.0,0.99096,3.07,0.72,11.3
5495,5495,5,6.6,0.410,0.31,1.6,0.042,18.0,101.0,0.99195,3.13,0.41,10.5


In [5]:
# 결과값을 저장할 VIF라는 이름의 리스트 생성
vif = []

# values atribution(속성) 이용해 train 데이터의 값만 추출
train_val = train.values

#  variance_inflation_factor 함수에 인자로 train_val (train 데이터의 값)과 인덱스 번호 지정  
# for문 이용
for i in range(len(train.columns)):
  vif.append(variance_inflation_factor(train_val, i))

In [6]:
vif_dataframe = pd.DataFrame()
vif_dataframe['columns'] = train.columns
vif_dataframe['VIF'] = vif
vif_dataframe

,columns,VIF
0,index,4.007175
1,quality,64.107432
2,fixed acidity,59.149187
3,volatile acidity,9.610687
4,citric acid,9.273444
5,residual sugar,3.616414
6,chlorides,5.715981
7,free sulfur dioxide,8.420883
8,total sulfur dioxide,14.833789
9,density,945.323991


- VIF는 변수간의 다중공선성을 진단하는 수치이며 범위 1부터 무한대
- 통계학에서는 VIF 값이 10이상이면 해당 변수가 다중공선성이 잇는 것으로 판단
- VIF를 구하는 수식은 VIFk = 1 / (1 - Rj2)

여기서 VIFk 는 k번째 변수의 VIF 값을 의미하고, Rj2 는 회귀분석에서 사용하는 결정계수